In [ ]:
import numpy as np
import pandas as pd
import time
import random
import os
import re
import json
import ast
import seaborn as sns
from helper_functions_assignment2 import*

In [ ]:
folder_path = "data/inst_competition/"
problem_instances = [file.replace('.txt', '') for file in os.listdir(folder_path) if not file == "test.txt"]

# ALNS

### run for all best configs from SMAC3

In [ ]:
# read in the file with the best configurations
best_configs = 'ALNS_smac_results.txt'
with open(best_configs, 'r') as file:
    file_content = file.read()

# Extract configurations using regular expressions
config_pattern = r"Configuration\(values=\{([^}]*)\}\)"
config_values = re.findall(config_pattern, file_content)

# Convert configurations to a list of dictionaries
config_dicts = []
for config_val in config_values:
    config_dict = ast.literal_eval('{' + config_val + '}')
    config_dicts.append(config_dict)

In [ ]:
test_results = []
for c in config_dicts:
    print(c)
    for p in problem_instances:
        print(p)
        path = folder_path+p+".txt"
        node_impact_orig, node_degree_orig, plex_assignment, edges_n1, edges_n2, edge_weights, edge_assignment_orig, s, n, m = create_problem_instance(path)

        start = time.time()
        plex_assignment = np.random.choice(10, n)
        number_of_phases = round(4000 / c["iterations_per_phase"])
        repair_solution(node_impact_orig, node_degree_orig, plex_assignment, edge_weights, edge_assignment_orig, s)

        
        plex_assignment, edge_assignment, node_impact, node_degree, score = ALNS(min_weight = c["min_weight"],
                                                                                 reaction_factor = c["reaction_factor"], 
                                                                                 iterations_per_phase = c["iterations_per_phase"], 
                                                                                 number_of_phases = number_of_phases,
                                                                                 node_impact = node_impact_orig, 
                                                                                 node_degree = node_degree_orig, 
                                                                                 edge_assignment = edge_assignment_orig, 
                                                                                 edge_weights = edge_weights, 
                                                                                 plex_assignment = plex_assignment, 
                                                                                 s = s, 
                                                                                 trajectory = False)

        test_results.append({"Instance": p,
                             "min_weight": c["min_weight"], 
                             "reaction_factor": c["reaction_factor"], 
                             "iterations_per_phase": c["iterations_per_phase"], 
                             "number_of_phases": number_of_phases,
                             "edge_assignment": edge_assignment,
                             "baseline": sum(node_impact_orig)/2,
                             "score": score, 
                             "runtime": time.time()-start})

In [ ]:
results = pd.DataFrame(test_results)
results["instance_ID"] = results["Instance"].str[4:7]
results

### get the rows with best results for each instance

In [ ]:
results.loc[results.groupby('Instance').score.idxmin()]

### save best results

In [ ]:
for idx in results.groupby('Instance').score.idxmin():
    path = folder_path+ results.loc[idx].Instance+".txt"
    node_impact_orig, node_degree_orig, plex_assignment, edges_n1, edges_n2, edge_weights, edge_assignment_orig, s, n, m = create_problem_instance(path)
    
    write_solution(results.loc[idx].edge_assignment, 
                   results.loc[idx].Instance, 
                   edge_weights, 
                   edges_n1, 
                   edges_n2, 
                   algorithm = "ALNS")